In [1]:
from sage.rings.ring import Field

In [4]:
[p for p in dir(Field) if p not in dir(Parent)]

['__fraction_field',
 '__ideal_monoid',
 '__iter__',
 '__len__',
 '__pow__',
 '__rpow__',
 '__rtruediv__',
 '__rxor__',
 '__truediv__',
 '__xor__',
 '_an_element_impl',
 '_coerce_',
 '_coerce_c',
 '_coerce_impl',
 '_coerce_try',
 '_default_category',
 '_gens',
 '_ideal_class_',
 '_latex_names',
 '_list',
 '_one_element',
 '_pseudo_fraction_field',
 '_random_nonzero_element',
 '_unit_ideal',
 '_zero_element',
 '_zero_ideal',
 'algebraic_closure',
 'base_extend',
 'class_group',
 'content',
 'derivation',
 'derivation_module',
 'divides',
 'epsilon',
 'extension',
 'fraction_field',
 'frobenius_endomorphism',
 'gcd',
 'gen',
 'gens',
 'ideal',
 'ideal_monoid',
 'integral_closure',
 'is_commutative',
 'is_field',
 'is_integral_domain',
 'is_integrally_closed',
 'is_noetherian',
 'is_prime_field',
 'is_subring',
 'krull_dimension',
 'localization',
 'ngens',
 'one',
 'order',
 'prime_subfield',
 'principal_ideal',
 'quo',
 'quotient',
 'quotient_ring',
 'random_element',
 'unit_ideal',
 'z

In [5]:
from sage.structure.unique_representation import UniqueRepresentation
class MyFrac(UniqueRepresentation, Field):
    def __init__(self, base):
        if base not in IntegralDomains():
            raise ValueError("%s is no integral domain"%base)
        Field.__init__(self, base)
    def _repr_(self):
        return "NewFrac(%s)"%repr(self.base())
    def base_ring(self):
        return self.base().base_ring()
    def characteristic(self):
        return self.base().characteristic()

In [7]:
MyFrac(ZZ['x'])

NewFrac(Univariate Polynomial Ring in x over Integer Ring)

In [9]:
from sage.structure.element import FieldElement

In [10]:
FieldElement(ZZ)

Generic element of a structure

In [40]:
class MyElement(FieldElement):
    def __init__(self, parent,n,d=None):
        B = parent.base()
        if d is None:
            d = B.one()
        if n not in B or d not in B:
            raise ValueError("Numerator and denominator must be elements of %s"%B)
        # Numerator and denominator should not just be "in" B,
        # but should be defined as elements of B
        d = B(d)
        n = B(n)
        if d==0:
            raise ZeroDivisionError("The denominator must not be zero")
        if d<0:
            self.n = -n
            self.d = -d
        else:
            self.n = n
            self.d = d
        FieldElement.__init__(self,parent)
    def numerator(self):
        return self.n
    def denominator(self):
        return self.d
    def _repr_(self):
        return "(%s):(%s)"%(self.n,self.d)
    def _richcmp_(self, other, op):
        from sage.structure.richcmp import richcmp
        return richcmp(self.n*other.denominator(), other.numerator()*self.d, op)
    def _add_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()+self.d*other.numerator(), D)
    def _sub_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()-self.d*other.numerator(),D)
    def _mul_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.numerator(), self.d*other.denominator())
    def _div_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.denominator(), self.d*other.numerator())

In [43]:
P = MyFrac(ZZ)
a = MyElement(P,3,4)
b = MyElement(P,1,2)
print(a,b,a+b,a-b,a*b,a/b)

(3):(4) (1):(2) (10):(8) (2):(8) (3):(8) (6):(4)


In [39]:
from sage.groups.additive_abelian.additive_abelian_group import AdditiveAbelianGroup, AdditiveAbelianGroupElement

In [31]:
??AdditiveAbelianGroup_class

Object `AdditiveAbelianGroup_class` not found.


In [46]:
G = AdditiveAbelianGroup([3])
g = G.0
C = G.category()

In [48]:
C.parent_class(), C.element_class()

(<sage.categories.modules.Modules.parent_class object at 0x6ffe1f857b10>,
 <sage.categories.modules.Modules.element_class object at 0x6ffe1f857b90>)

In [51]:
from sage.misc.abstract_method import abstract_methods_of_class

In [52]:
abstract_methods_of_class(C.element_class)['optional']

['_add_']

In [53]:
abstract_methods_of_class(C.element_class)['required']

['__bool__']

In [55]:
from sage.categories.quotient_fields import QuotientFields

In [56]:
class MyFrac(MyFrac):
    def __init__(self, base, category=None):
        if base not in IntegralDomains():
            raise ValueError("%s is no integral domain" % base)
        Field.__init__(self, base, category=category or QuotientFields())

In [58]:
P = MyFrac(ZZ)
type(P)

<class '__main__.MyFrac_with_category'>

In [59]:
isinstance(P,MyFrac)

True

In [60]:
isinstance(P,QuotientFields().parent_class)

True

In [61]:
class MyFrac(MyFrac):
    Element = MyElement

In [64]:
P = MyFrac(ZZ)
P(1), P(2,3)

((1):(1), (2):(3))

In [65]:
P.zero()

(0):(1)

In [66]:
a = MyElement(P, 9, 4)
b = MyElement(P, 1, 2)
c = MyElement(P, -1, 2)
P.sum([a,b,c])

(36):(16)

In [67]:
a^3

(729):(64)

In [68]:
P.__class__.element_class

In [69]:
P.element_class

<class '__main__.MyFrac_with_category.element_class'>

In [70]:
type(P.element_class)

<class 'sage.structure.dynamic_class.DynamicInheritComparisonMetaclass'>

In [71]:
issubclass(P.element_class, MyElement)

True

In [72]:
issubclass(P.element_class,P.category().element_class)

True

In [73]:
type(P(2,3))

<class '__main__.MyFrac_with_category.element_class'>

In [74]:
type(a)

<class '__main__.MyElement'>

In [75]:
isinstance(a,P.element_class)

False

In [76]:
type(P.sum([a,b,c]))

<class '__main__.MyFrac_with_category.element_class'>

In [77]:
a; a.factor(); P(6,4).factor()

2^-1 * 3

In [78]:
class MyFrac(MyFrac):
    def _element_constructor_(self, *args, **kwds):
        if len(args)!=1:
            return self.element_class(self, *args, **kwds)
        x = args[0]
        try:
            P = x.parent()
        except AttributeError:
            return self.element_class(self, x, **kwds)
        if P in QuotientFields() and P != self.base():
            return self.element_class(self, x.numerator(), x.denominator(), **kwds)
        return self.element_class(self, x, **kwds)

In [79]:
P = MyFrac(ZZ)
print(P(2), P(2,3), P(3/4))

(2):(1) (2):(3) (3):(4)


In [80]:
sage: class MyFrac(MyFrac):
    def _coerce_map_from_(self, S):
        if self.base().has_coerce_map_from(S):
            return True
        if S in QuotientFields():
            if self.base().has_coerce_map_from(S.base()):
                return True
            if hasattr(S,'ring_of_integers') and self.base().has_coerce_map_from(S.ring_of_integers()):
                return True

In [84]:
P = MyFrac(QQ['x'])
P.has_coerce_map_from(ZZ['x']), P.has_coerce_map_from(Frac(ZZ['x'])), P.has_coerce_map_from(QQ)

(True, True, True)

In [83]:
3/4+P(2)+ZZ['x'].gen(), (P(2)+ZZ['x'].gen()).parent() is P

((4*x + 11):(4), True)

In [85]:
1 in P

True

In [87]:
P = MyFrac(ZZ)
1/2+P(2,3)+1

(13):(6)

In [88]:
P(1/2) + ZZ['x'].gen(), (P(1/2) + ZZ['x'].gen()).parent() is P['x']

((1):(1)*x + (1):(2), True)

In [90]:
Poly,R = QQ['x'].construction()
Poly,R

(Poly[x], Rational Field)

In [91]:
Fract,R = QQ.construction()
Fract,R

(FractionField, Integer Ring)

In [92]:
Poly(QQ) is QQ['x']

True

In [93]:
Poly(ZZ) is ZZ['x']

True

In [94]:
Poly(P) is P['x']

True

In [95]:
Fract(QQ['x'])

Fraction Field of Univariate Polynomial Ring in x over Rational Field

In [96]:
Poly.domain()

Category of rings

In [97]:
Poly.codomain()

Category of rings

In [98]:
Fract.domain()

Category of integral domains

In [99]:
Fract.codomain()

Category of fields

In [100]:
Poly*Fract

Poly[x](FractionField(...))

In [101]:
(Poly*Fract)(ZZ) is QQ['x']

True

In [102]:
((Poly*Fract)(ZZ)).coerce_map_from(ZZ)

Composite map:
  From: Integer Ring
  To:   Univariate Polynomial Ring in x over Rational Field
  Defn:   Natural morphism:
          From: Integer Ring
          To:   Rational Field
        then
          Polynomial base injection morphism:
          From: Rational Field
          To:   Univariate Polynomial Ring in x over Rational Field

In [103]:
(Fract*Poly)(ZZ)

Fraction Field of Univariate Polynomial Ring in x over Integer Ring

In [104]:
from sage.categories.pushout import pushout
pushout(Fract(ZZ),Poly(ZZ))

Univariate Polynomial Ring in x over Rational Field

In [105]:
Fract.rank, Poly.rank

(5, 9)

In [106]:
Fract.pushout(Poly), Poly.pushout(Fract)

(Poly[x](FractionField(...)), Poly[x](FractionField(...)))

In [113]:
from sage.categories.pushout import ConstructionFunctor
class MyFracFunctor(ConstructionFunctor):
    rank = 5
    def __init__(self, args=None, kwds=None):
        self.args = args or ()
        self.kwds = kwds or {}
        ConstructionFunctor.__init__(self, IntegralDomains(), Fields())
    def _apply_functor(self, R):
        return MyFrac(R,*self.args,**self.kwds)
    def merge(self, other):
        if isinstance(other, (type(self), sage.categories.pushout.FractionField)):
             return self

In [115]:
MyFracFunctor()(ZZ)

NewFrac(Integer Ring)

In [116]:
MyFracFunctor().merge(Fract)

MyFracFunctor

In [117]:
class MyFrac(MyFrac):
    def construction(self):
        return MyFracFunctor(self._reduction[1][1:], self._reduction[2]), self.base()

In [118]:
MyFrac(ZZ['x']).construction()

(MyFracFunctor, Univariate Polynomial Ring in x over Integer Ring)

In [119]:
pushout(MyFrac(ZZ['x']), Frac(QQ['x']))

NewFrac(Univariate Polynomial Ring in x over Rational Field)

In [120]:
from sage.misc.abstract_method import abstract_methods_of_class

In [121]:
abstract_methods_of_class(QuotientFields().parent_class)['optional']

[]

In [122]:
abstract_methods_of_class(QuotientFields().parent_class)['required']

['__contains__']

In [123]:
abstract_methods_of_class(QuotientFields().element_class)['optional']

['_add_', '_mul_']

In [124]:
abstract_methods_of_class(QuotientFields().element_class)['required']

['__bool__', 'denominator', 'numerator']

In [125]:
class Foo(Parent):
    Element = sage.structure.element.Element
    def __init__(self):
        Parent.__init__(self, category=QuotientFields())

In [127]:
Bar = Foo()
bar = Bar.element_class(Bar)
bar._test_not_implemented_methods()

AssertionError: Not implemented method: denominator

In [128]:
[t for t in dir(QuotientFields().parent_class) if t.startswith('_test_')]

['_test_additive_associativity',
 '_test_an_element',
 '_test_associativity',
 '_test_cardinality',
 '_test_characteristic',
 '_test_characteristic_fields',
 '_test_construction',
 '_test_distributivity',
 '_test_divides',
 '_test_elements',
 '_test_elements_eq_reflexive',
 '_test_elements_eq_symmetric',
 '_test_elements_eq_transitive',
 '_test_elements_neq',
 '_test_euclidean_degree',
 '_test_fraction_field',
 '_test_gcd_vs_xgcd',
 '_test_one',
 '_test_prod',
 '_test_quo_rem',
 '_test_some_elements',
 '_test_zero',
 '_test_zero_divisors']

In [131]:
P = MyFrac(ZZ['x'])
TestSuite(P).run(verbose =True)

running ._test_additive_associativity() . . . pass
running ._test_an_element() . . . pass
running ._test_associativity() . . . pass
running ._test_cardinality() . . . pass
running ._test_category() . . . pass
running ._test_characteristic() . . . pass
running ._test_characteristic_fields() . . . pass
running ._test_construction() . . . pass
running ._test_distributivity() . . . pass
running ._test_divides() . . . pass
running ._test_elements() . . .
  Running the test suite of self.an_element()
  running ._test_category() . . . pass
  running ._test_eq() . . . pass
  running ._test_new() . . . pass
  running ._test_nonzero_equal() . . . pass
  running ._test_not_implemented_methods() . . . pass
  running ._test_pickling() . . . pass
  pass
running ._test_elements_eq_reflexive() . . . pass
running ._test_elements_eq_symmetric() . . . pass
running ._test_elements_eq_transitive() . . . pass
running ._test_elements_neq() . . . pass
running ._test_eq() . . . pass
running ._test_euclidean_de

In [132]:
class MyFrac(MyFrac):
    def _an_element_(self):
        a = self.base().an_element()
        b = self.base_ring().an_element()
        if (a+b)!=0:
            return self(a)**2/(self(a+b)**3)
        if b != 0:
            return self(a)/self(b)**2
        return self(a)**2*self(b)**3
    def some_elements(self):
        return [self.an_element(),self(self.base().an_element()),self(self.base_ring().an_element())]

In [133]:
P = MyFrac(ZZ['x'])
P.an_element(); P.some_elements()

[(x^2):(x^3 + 3*x^2 + 3*x + 1), (x):(1), (1):(1)]

In [136]:
from sage.categories.category import Category
class QuotientFieldsWithTest(Category): # do *not* inherit from QuotientFields, but ...
    def super_categories(self):
        return [QuotientFields()]       # ... declare QuotientFields as a super category!
    class ParentMethods:
        pass
    class ElementMethods:
        def _test_factorisation(self, **options):
            P = self.parent()
            assert self == P.prod([P(b)**e for b,e in self.factor()])

In [137]:
P = MyFrac(ZZ['x'], category=QuotientFieldsWithTest())
P.category()

Category of quotient fields with test

In [138]:
P.an_element()._test_factorisation

<bound method QuotientFieldsWithTest.ElementMethods._test_factorisation of (x^2):(x^3 + 3*x^2 + 3*x + 1)>

In [139]:
P.an_element().factor()

(x + 1)^-3 * x^2

In [140]:
TestSuite(P).run(verbose=True)

running ._test_additive_associativity() . . . pass
running ._test_an_element() . . . pass
running ._test_associativity() . . . pass
running ._test_cardinality() . . . pass
running ._test_category() . . . pass
running ._test_characteristic() . . . pass
running ._test_characteristic_fields() . . . pass
running ._test_construction() . . . pass
running ._test_distributivity() . . . pass
running ._test_divides() . . . pass
running ._test_elements() . . .
  Running the test suite of self.an_element()
  running ._test_category() . . . pass
  running ._test_eq() . . . pass
  running ._test_factorisation() . . . pass
  running ._test_new() . . . pass
  running ._test_nonzero_equal() . . . pass
  running ._test_not_implemented_methods() . . . pass
  running ._test_pickling() . . . pass
  pass
running ._test_elements_eq_reflexive() . . . pass
running ._test_elements_eq_symmetric() . . . pass
running ._test_elements_eq_transitive() . . . pass
running ._test_elements_neq() . . . pass
running ._test

In [1]:
# Importing base classes, ...
import sage
from sage.rings.ring import Field
from sage.structure.element import FieldElement
from sage.categories.category import Category
# ... the UniqueRepresentation tool,
from sage.structure.unique_representation import UniqueRepresentation
# ... some categories, and ...
from sage.categories.fields import Fields
from sage.categories.quotient_fields import QuotientFields
from sage.categories.integral_domains import IntegralDomains
# construction functors
from sage.categories.pushout import ConstructionFunctor

# Fraction field elements
class MyElement(FieldElement):
    def __init__(self, parent, n, d=None):
        if parent is None:
            raise ValueError("The parent must be provided")
        B = parent.base()
        if d is None:
            # The default denominator is one
            d = B.one()
        # verify that both numerator and denominator belong to the base
        if n not in B or d not in B:
            raise ValueError("Numerator and denominator must be elements of %s"%B)
        # Numerator and denominator should not just be "in" B,
        # but should be defined as elements of B
        d = B(d)
        n = B(n)
        # the denominator must not be zero
        if d==0:
            raise ZeroDivisionError("The denominator must not be zero")
        # normalize the denominator: WLOG, it shall be non-negative.
        if d<0:
            self.n = -n
            self.d = -d
        else:
            self.n = n
            self.d = d
        FieldElement.__init__(self,parent)

    # Methods required by the category of fraction fields:
    def numerator(self):
        return self.n
    def denominator(self):
        return self.d

    # String representation (single underscore!)
    def _repr_(self):
        return "(%s):(%s)"%(self.n,self.d)

    # Comparison: We can assume that both arguments are coerced
    # into the same parent, which is a fraction field. Hence, we
    # are allowed to use the denominator() and numerator() methods
    # on the second argument.
    def _richcmp_(self, other, op):
        from sage.structure.richcmp import richcmp
        return richcmp(self.n*other.denominator(), other.numerator()*self.d, op)

    # Arithmetic methods, single underscore. We can assume that both
    # arguments are coerced into the same parent.
    # We return instances of self.__class__, because self.__class__ will
    # eventually be a sub-class of MyElement.
    def _add_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()+self.d*other.numerator(),D)
    def _sub_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()-self.d*other.numerator(),D)
    def _mul_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.numerator(), self.d*other.denominator())
    def _div_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.denominator(), self.d*other.numerator())

# Inheritance from UniqueRepresentation implements the unique parent
# behaviour. Moreover, it implements pickling (provided that Python
# succeeds to look up the class definition).
class MyFrac(UniqueRepresentation, Field):
    # Implement the category framework for elements, which also
    # makes some basic conversions work.
    Element = MyElement

    # Allow to pass to a different category, by an optional argument
    def __init__(self, base, category=None):
        # Fraction fields only exist for integral domains
        if base not in IntegralDomains():
            raise ValueError("%s is no integral domain" % base)
        # Implement the category framework for the parent
        Field.__init__(self, base, category=category or QuotientFields())

    # Single-underscore method for string representation
    def _repr_(self):
        return "NewFrac(%s)"%repr(self.base())

    # Two methods that are implicitly used in some tests
    def base_ring(self):
        return self.base().base_ring()
    def characteristic(self):
        return self.base().characteristic()

    # Implement conversions. Do not override __call__!
    def _element_constructor_(self, *args, **kwds):
        if len(args)!=1:
            return self.element_class(self, *args, **kwds)
        x = args[0]
        try:
            P = x.parent()
        except AttributeError:
            return self.element_class(self, x, **kwds)
        if P in QuotientFields() and P != self.base():
            return self.element_class(self, x.numerator(), x.denominator(), **kwds)
        return self.element_class(self, x, **kwds)

    # Implement coercion from the base and from fraction fields
    # over a ring that coerces into the base
    def _coerce_map_from_(self, S):
        if self.base().has_coerce_map_from(S):
            return True
        if S in QuotientFields():
            if self.base().has_coerce_map_from(S.base()):
                return True
            if hasattr(S,'ring_of_integers') and self.base().has_coerce_map_from(S.ring_of_integers()):
                return True
    # Tell how this parent was constructed, in order to enable pushout constructions
    def construction(self):
        return MyFracFunctor(), self.base()

    # return some elements of this parent
    def _an_element_(self):
        a = self.base().an_element()
        b = self.base_ring().an_element()
        if (a+b)!=0:
            return self(a)**2/(self(a+b)**3)
        if b != 0:
            return self(a)/self(b)**2
        return self(a)**2*self(b)**3
    def some_elements(self):
        return [self.an_element(),self(self.base().an_element()),self(self.base_ring().an_element())]


# A construction functor for our implementation of fraction fields
class MyFracFunctor(ConstructionFunctor):
    # The rank is the same for Sage's original fraction field functor
    rank = 5
    def __init__(self):
        # The fraction field construction is a functor
        # from the category of integral domains into the category of
        # fields
        # NOTE: We could actually narrow the codomain and use the
        # category QuotientFields()
        ConstructionFunctor.__init__(self, IntegralDomains(), Fields())
    # Applying the functor to an object. Do not override __call__!
    def _apply_functor(self, R):
        return MyFrac(R)
    # Note: To apply the functor to morphisms, implement
    #       _apply_functor_to_morphism

    # Make sure that arithmetic involving elements of Frac(R) and
    # MyFrac(R) works and yields elements of MyFrac(R)
    def merge(self, other):
        if isinstance(other, (type(self), sage.categories.pushout.FractionField)):
            return self

# A quotient field category with additional tests.
# Notes:
# - Category inherits from UniqueRepresentation. Hence, there
#   is only one category for given arguments.
# - Since QuotientFieldsWithTest is a singleton (there is only
#   one instance of this class), we could inherit from
#   sage.categories.category_singleton.Category_singleton
#   rather than from sage.categories.category.Category
class QuotientFieldsWithTest(Category):
    # Our category is a sub-category of the category of quotient fields,
    # by means of the following method.
    def super_categories(self):
        return [QuotientFields()]

    # Here, we could implement methods that are available for
    # all objects in this category.
    class ParentMethods:
        pass

    # Here, we add a new test that is available for all elements
    # of any object in this category.
    class ElementMethods:
        def _test_factorisation(self, **options):
            P = self.parent()
            # The methods prod() and factor() are inherited from
            # some other categories.
            assert self == P.prod([P(b)**e for b,e in self.factor()])

In [2]:
P = MyFrac(ZZ); P

NewFrac(Integer Ring)